In [1]:
import logging
import transformers
import numpy as np
import pandas as pd
import tensorflow as tf
from transformers import TFBertPreTrainedModel
from transformers import TFBertMainLayer
from transformers import BertTokenizer
from transformers.modeling_tf_utils import TFQuestionAnsweringLoss
from transformers.modeling_tf_utils import TFTokenClassificationLoss
from transformers.modeling_tf_utils import get_initializer
from transformers.modeling_tf_utils import keras_serializable
from transformers.modeling_tf_utils import shape_list
from sklearn.metrics import roc_auc_score

In [14]:
logging.basicConfig(level=logging.INFO)

In [3]:
(tf.__version__, transformers.__version__)

('2.2.0', '3.1.0')

In [5]:
def convert_example_to_feature(review, max_len):
  
  # combine step for tokenization, WordPiece vector mapping, adding special tokens as well as truncating reviews longer than the max length
    return tokenizer.encode_plus(review, 
                add_special_tokens = True, # add [CLS], [SEP]
                max_length = max_len,      # max length of the text that can go to BERT
                pad_to_max_length = True,  # add [PAD] tokens
                return_attention_mask = True, # add attention mask to not focus on pad tokens
                truncation=True)


# map to the expected input to TFBertForSequenceClassification, see here 
def map_example_to_dict(input_ids, attention_masks, token_type_ids, label):
    x = {"input_ids": input_ids, 
         "token_type_ids": token_type_ids, 
         "attention_mask": attention_masks}
    return x, label


def encode_examples(data, limit=-1):
    # prepare list, so that we can build up final TensorFlow dataset from slices.
    input_ids_list = []
    token_type_ids_list = []
    attention_mask_list = []
    label_list = []
    if (limit > 0):
        ds = ds.take(limit)
    
    for (i, row) in enumerate(ds.values):
#     for index, row in ds.iterrows():
#         review = row["text"]
#         label = row["y"]
        review = row[1]
        label = list(row[2:])
        bert_input = convert_example_to_feature(review)
  
        input_ids_list.append(bert_input['input_ids'])
        token_type_ids_list.append(bert_input['token_type_ids'])
        attention_mask_list.append(bert_input['attention_mask'])
        label_list.append(label)
    return tf.data.Dataset.from_tensor_slices(
        (input_ids_list, attention_mask_list, token_type_ids_list, label_list)
        ).map(map_example_to_dict)

    
class TFBertForMultilabelClassification(TFBertPreTrainedModel):

    def __init__(self, config, *inputs, **kwargs):
        super(TFBertForMultilabelClassification, self).__init__(config, *inputs, **kwargs)
        self.num_labels = config.num_labels

        self.bert = TFBertMainLayer(config, name='bert')
        self.dropout = tf.keras.layers.Dropout(config.hidden_dropout_prob)
        self.classifier = tf.keras.layers.Dense(config.num_labels,
                                                kernel_initializer=get_initializer(config.initializer_range),
                                                name='classifier',
                                                activation='sigmoid')

    def call(self, inputs, **kwargs):
        outputs = self.bert(inputs, **kwargs)

        pooled_output = outputs[1]

        pooled_output = self.dropout(pooled_output, training=kwargs.get('training', False))
        logits = self.classifier(pooled_output)

        outputs = (logits,) + outputs[2:]  # add hidden states and attention if they are here

        return outputs  # logits, (hidden_states), (attentions)


def measure_auc(label, pred):
    auc = [roc_auc_score(label[:,i],pred[:,i]) for i in list(range(2))]
    return pd.DataFrame({"label_name":["unsafe","safe"],"auc":auc})

def load_data(filename):
    with open(filename, "r") as f:
        data = f.read().splitlines()
    return [s.split("\t") for s in data]

In [9]:
train_path = "data/train.test.txt" # 数据路径
test_path = "data/test.txt" # 数据路径
model_path = "data/" #模型路径，建议预先下载(https://huggingface.co/bert-base-uncased#)

max_len = 128
batch_size = 8
learning_rate = 2e-5
number_of_epochs = 2
num_classes = 2 # 类别数

In [7]:
data = load_data(train_path)

In [8]:
split_n = int(len(data) * 0.9)
split_n

4500

In [10]:
train_data = data[:split_n]
val_data = data[split_n:]
test_data = load_data(test_path)

In [11]:
train_data[:2]

[['不想 花钱 不行 喜 听 男 冒险 晚上 撒娇 找 文本 撒娇 说 想 选 失眠 空间 想念 两个 画面 眼泪 憔悴 洒满 心碎 大雨 天成 告 受不了 爱 可爱 喵 早上好 聊聊天 好不好 可爱 聊天 天 捶 胸口 喵 不用 滚回来 妈 小拳头 胸口 召唤 一拳 肯定 死哦 偷 陪陪 超 想 哭 捶 胸口 大坏蛋 说 胸口 你好 讨厌 抱抱 嘤 小球 捶 胸口 大坏蛋 打死 小熊 嘤 陪 小拳头 捶 年纪 想 眼 领 说 咿呀 说 瞬间 开心 样子 心 懂 说 爱 淡 梦 远 开心 数着 舍 爱 感觉 太 深刻 喜欢 太 靠近 我心 空 请 香港 工作 真心 感觉 成 大意 怀里 回头 活好 可爱 当初 说 帅哥 未来 再现 沧桑 温柔 感受 年 八年级 太 玩伴 做 撒娇 回头一看 弄 年 十 二十 还好 巴拉巴 拉 放 分钟 诱惑 来到 梅 英 张 晨 主持 男人 爱 伤害 小可爱',
  '0'],
 ['疏导 进 金子 版 类型 管子 里 为啥 射 远 破解 说 金 说 那根筋 扯出来 事儿 扯出来 啥事 把头 两个 蛋壳 去掉 那根 尖 肉 去掉 两根 睾丸 两个 睾丸 青紫 拔掉 行 讲 说 通俗 点 孩子 解答 子宫 效果 受孕 没事 聊天 生 可不 卡 太 大型 那天 月黑风高 月 打算 找 草丛 查 发现 远处 树荫下 传来 一阵 阴冷 哭泣声 娘 对视 一眼 一把 屎 一把 尿 孩子 找 真 听 恩 麦片 太难受 本来 讲 家庭 做 再三 唉呀 娘 想想 简单 事 演 说 正 彩旗 外 外面 家里 红旗 倒 外面 彩旗 飘飘 听到 说出 金子 受孕 操 角色 先 受孕 先 受 受孕 小姑娘 商城 保 双重 牛皮 屁股 擦 太强大 不行 先 想 小姑娘 双重 可不 爹 弟弟 说 听不懂 诶 爹 无关 围观 下面 围观 聊 这么久 普及 生 学 生物学 生理学 普及 时 关 受孕 一种 可惜 大学 听 说 大学 生物 老师 大学生 要说 好看 现场表演 生育 到时候 我去 回来',
  '1']]

In [12]:
test_data[:2]

[['二十三 选项 选 麻烦 成功 场 两个 高级 宝箱 豪迈 流 卖 冒险 国家 说 鬼子 筛选 讲 年 弟弟 妹妹 弟弟 一串 英文 简单 说 当成 买 牙牙 乐 接 大冒险 关爱 喜欢 男人 那天 宝贝 喜欢 说 果子 天涯 听 第一 女 扳手 搬 弯 前缀 简单 狐狸 句 话 说 大伙 心头 喜欢 男人 老子 男人 老子 男人 老子 男人 冒险 环节 卖 南方 说 老张 油腻 中年 大叔 男人 行 卖 冒险 说 卖 冒险 说 操 南方 说 麦 留下 联系方式 走 四号 卖 延安 乐 街 大冒险 楼 五号 卖 宇 哥哥 开 卖 有没有 要说 放 点 狠 话 放 狠 话 这么久 一只 一点 游戏 体验 说话 毛病 冒险 进 面子 台阶 爽 心服口服 翻 几楼 说 几楼 大队 那种 刀法 那种 想 说句话 等会儿 帮 早上 果子 说 点 解放 问 三字经 简 不见 不想 娶 做 四个 私下 找 活儿 渣 女 你好 卖 声 差 口味 冉 名 大腿 做做 冒险 寻思 妹妹 小甜甜 小朋友 徒弟 老三 魅力 下降 办法 老三 小甜甜 小小 凯 先抱 叫卖 帮 徒弟 卖身 差 样子 南方 干嘛 没意思 退 说',
  '0'],
 ['何日 里 花花绿绿 世界 里 喜欢 看不到 地方 安然无恙 愿 冬天 永远 缺 太阳 明天 雨 风霜 愿 未来 天 热 哇哇 美好 甜蜜 柴米 油盐酱醋 祝 春风 带给 空气 温柔 世界 里 眼睛 好脾气 我爱你 愿 岁 夕阳 想 嘴角 微微 上扬 天 白方 冬天 披 点 衣裳 入 戏 留 身旁 哇哇 看不到 地方 爱 无恙 愿 痛 永远 明天 岁 陪 夕阳 愿 想起 想要 愿 北方 冬天 提 点 影响 越过 新 报应 睡着 地面 报应 生活 里 惊喜 美丽 老婆 挺 喜欢 小宝贝 谢 兰 保密 市场 谢谢 小雨 不想 听歌 逝去 过往 回头 逝去 过往 真的 背着 行囊 随便 漫画 山 做 槟榔 算账 地上 梦 醒 寻找 美丽 风景 老人 病 中心 躺 床上 忘记 错误 想法 脑 里 付出 不想 名字 绝路 舒服 渴望 自由 像是 治不好 病理 太 信 静 圆心 女孩 家人 哥 原谅 说 逼 说 外婆 回来 变 贪心 神情 波 做 有福气 说',
  '0']]

In [15]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

In [ ]:
ds_train_encoded = encode_examples(train_data).shuffle(10000).batch(batch_size)

In [20]:

    # tokenizer
    
    # train dataset
    
    # val dataset
    ds_val_encoded = encode_examples(val_data).batch(batch_size)
    # test dataset
    ds_test_encoded = encode_examples(test_data).batch(batch_size)
    
    # model initialization
    model = TFBertForMultilabelClassification.from_pretrained(model_path, num_labels=num_classes)
    # optimizer Adam recommended
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate,epsilon=1e-08, clipnorm=1)
    # we do not have one-hot vectors, we can use sparce categorical cross entropy and accuracy
    loss = tf.keras.losses.BinaryCrossentropy()
    metric = tf.keras.metrics.CategoricalAccuracy()
    model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

    # fit model
    bert_history = model.fit(ds_train_encoded, epochs=number_of_epochs, validation_data=ds_val_encoded)
    # evaluate val_set
    pred=model.predict(ds_val_encoded)[0]
    df_auc = measure_auc(val_data.iloc[:,2:].astype(np.float32).values,pred)
    print("val set mean column auc:",df_auc)

FileNotFoundError: [Errno 2] No such file or directory: 'train.csv'

In [18]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(
    num_words=None, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True,
    split=' ', char_level=False, oov_token=None, document_count=0)


In [19]:
tokenizer.document_count

0

In [20]:
tokenizer = BertTokenizer("data/assets/vocab.txt")

In [28]:
tokenizer.texts_to_sequences(["我","爱"])

[[], []]

In [32]:
?BertTokenizer.from_pretrained